<h2>SAXS data processing</h2>
Template notebook by Guillaume (gfreychet@bnl.gov, not active), development and analysis by Patryk (pwasik@bnl.gov)

In [1]:
import time
import os
import numpy as np
import fabio
import pandas as pd
import matplotlib.pyplot as plt
from smi_analysis import SMI_beamline

%matplotlib notebook

/Users/ianbillinge/micromamba/envs/smi_py311/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
from scipy import interpolate
from matplotlib import (transforms, colors, cm)
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, ScalarFormatter, LogLocator)
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colorbar import Colorbar as colorbar
pd.set_option('display.max_colwidth', None)

In [3]:
cmap = cm.viridis.copy()
cmap.set_bad(cmap.get_under(), 1.)

sci_formatter = ScalarFormatter(useMathText=True)
sci_formatter.set_scientific(True)
sci_formatter.set_powerlimits((-1, 1))

## Locate the files on your computer

In [4]:
cycle_id = '2023_3'
proposal_id = '313303_Billinge_02'

# path_data = os.path.join(os.sep, 'nsls2', 'data', 'smi', 'legacy', 'results',  'data',
#                          cycle_id, proposal_id, '1M')
# path_save = os.path.join(os.sep, 'nsls2', 'data', 'smi', 'legacy', 'results', 'analysis',
#                          cycle_id, proposal_id, 'reduced_saxs')
path_data = os.path.join(os.sep, 
                         '/Users/ianbillinge/Documents/yiplab/projects/resonant/2023-11-30-BM-12/313303_Billinge_01', 
                         '1M')
path_save = os.path.join(os.sep, 
                         '/Users/ianbillinge/Documents/yiplab/projects/resonant/2023-11-30-BM-12/313303_Billinge_01', 
                         'reduced_saxs')
if os.path.isdir(path_data):
    os.makedirs(path_save, exist_ok=True)

<h4>Get files</h4> and sort them by date

In [5]:
files = sorted([os.path.join(path_data, f) for f in os.listdir(path_data)
                if ('test' not in f) and ('.tif' in f) and ('wa0' not in f) and ('wa-0' not in f) and ('inair' not in f)])
files.sort(key=lambda x: os.path.getmtime(x), reverse=False)
files = pd.Series([os.path.split(path_to_file)[-1] for path_to_file in files])
files

0       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45000keV_wa20.0_sdd1.8m_bpm3.405_id433605_000000_SAXS.tif
1       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45101keV_wa20.0_sdd1.8m_bpm003.4_id433606_000000_SAXS.tif
2       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45200keV_wa20.0_sdd1.8m_bpm3.399_id433607_000000_SAXS.tif
3       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45300keV_wa20.0_sdd1.8m_bpm3.396_id433608_000000_SAXS.tif
4       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45400keV_wa20.0_sdd1.8m_bpm3.393_id433609_000000_SAXS.tif
                                                    ...                                              
1059        IB_DIPA_RbBr_aq_phase-a_pos1_13.53500keV_wa40.0_sdd1.8m_bpm3.275_id435194_000000_SAXS.tif
1060        IB_DIPA_RbBr_aq_phase-a_pos1_13.54000keV_wa40.0_sdd1.8m_bpm3.297_id435195_000000_SAXS.tif
1061        IB_DIPA_RbBr_aq_phase-a_pos1_13.54501keV_wa40.0_sdd1.8m_bpm3.282_id435196_000000_SAXS.tif
1062        IB_DIPA_RbBr_aq_phase-a_pos1_13.55000keV_wa40.0_sdd1.8m_bpm3.287_id435

<h4>Sample names</h4>

In [6]:
samples = files[files.str.contains('temp')].str.split('temp', expand=True).iloc[:, 0].unique()
#samples = pd.Series([sample[:16] for sample in samples]).unique()
samples = pd.Series([s[3:] for s in samples])
samples

IndexError: single positional indexer is out-of-bounds

In [7]:
samples2 = files[~files.str.contains('16.1keV')]
#samples2 = samples2[samples2.str.contains('000001')].str.split('000001', expand=True).iloc[:, 0].unique()
samples2

0       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45000keV_wa20.0_sdd1.8m_bpm3.405_id433605_000000_SAXS.tif
1       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45101keV_wa20.0_sdd1.8m_bpm003.4_id433606_000000_SAXS.tif
2       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45200keV_wa20.0_sdd1.8m_bpm3.399_id433607_000000_SAXS.tif
3       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45300keV_wa20.0_sdd1.8m_bpm3.396_id433608_000000_SAXS.tif
4       IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45400keV_wa20.0_sdd1.8m_bpm3.393_id433609_000000_SAXS.tif
                                                    ...                                              
1059        IB_DIPA_RbBr_aq_phase-a_pos1_13.53500keV_wa40.0_sdd1.8m_bpm3.275_id435194_000000_SAXS.tif
1060        IB_DIPA_RbBr_aq_phase-a_pos1_13.54000keV_wa40.0_sdd1.8m_bpm3.297_id435195_000000_SAXS.tif
1061        IB_DIPA_RbBr_aq_phase-a_pos1_13.54501keV_wa40.0_sdd1.8m_bpm3.282_id435196_000000_SAXS.tif
1062        IB_DIPA_RbBr_aq_phase-a_pos1_13.55000keV_wa40.0_sdd1.8m_bpm3.287_id435

<h3>Sample names based on reg expressions</h3>

In [8]:
#test = files[files.str.contains('wa20')].str.split(r'\d{0,4}.\d{0,2}eV', expand=True).iloc[:, 0].unique()
samples = files[files.str.contains('wa40')].str.split(r'\d{0,6}.\d{0,6}keV', expand=True).iloc[:, 0].unique()

#test2 = pd.Series([s for s in samples if 'WAXS' not in s])
#test3 = files[files.str.contains('bpm')].str.split('wa20', expand=True).iloc[:, 0].unique()
#test3 = pd.Series([sample[:sample.find('eV_') - 7] for sample in samples]).unique()
#test3 = pd.Series(test3)
#test4 = pd.concat([test2, test3])
#samples = test
samples = pd.Series(samples)
samples

0                    IB_kaptom_pos1_
1        IB_LiBr_sol_no_solv-b_pos1_
2        IB_NaBr_sol_no_solv-b_pos1_
3         IB_KBr_sol_no_solv-b_pos1_
4        IB_RbBr_sol_no_solv-b_pos1_
5                   IB_water-a_pos1_
6     IB_DIPA_LiBr_org_phase-a_pos1_
7     IB_DIPA_RbBr_org_phase-a_pos1_
8      IB_DIPA_NaBr_aq_phase-a_pos1_
9       IB_DIPA_KBr_aq_phase-a_pos1_
10     IB_DIPA_RbBr_aq_phase-a_pos1_
dtype: object

<h3>Create lists for averaging SAXS</h3>detector images at the same energy but different WAXS angle

In [9]:
sample = samples.iloc[0]
matches = files[files.str.contains(f'{sample}')]# + r'\d{0,6}.\d{0,2}keV')]
matches

51     IB_kaptom_pos1_13.45000keV_wa20.0_sdd1.8m_bpm3.415_id433726_000000_SAXS.tif
52     IB_kaptom_pos1_13.45500keV_wa20.0_sdd1.8m_bpm3.421_id433727_000000_SAXS.tif
53     IB_kaptom_pos1_13.46000keV_wa20.0_sdd1.8m_bpm3.412_id433728_000000_SAXS.tif
54     IB_kaptom_pos1_13.46099keV_wa20.0_sdd1.8m_bpm3.404_id433729_000000_SAXS.tif
55     IB_kaptom_pos1_13.46201keV_wa20.0_sdd1.8m_bpm3.401_id433730_000000_SAXS.tif
                                          ...                                     
138    IB_kaptom_pos1_13.53000keV_wa40.0_sdd1.8m_bpm3.311_id433813_000000_SAXS.tif
139    IB_kaptom_pos1_13.53500keV_wa40.0_sdd1.8m_bpm3.281_id433814_000000_SAXS.tif
140    IB_kaptom_pos1_13.54000keV_wa40.0_sdd1.8m_bpm003.3_id433815_000000_SAXS.tif
141    IB_kaptom_pos1_13.54501keV_wa40.0_sdd1.8m_bpm3.276_id433816_000000_SAXS.tif
142    IB_kaptom_pos1_13.55000keV_wa40.0_sdd1.8m_bpm3.287_id433817_000000_SAXS.tif
Length: 92, dtype: object

In [10]:
#locations = pd.Series([f[f.find('loc') + 3 : f.find('loc') + 4] for f in matches]).unique()
locations = pd.Series([f[f.find('_SAXS.tif') - 6 : f.find('_SAXS.tif')] for f in matches]).unique()
#locations = pd.Series([f[f.find('-x') + 0 : f.find('_16.10keV')] for f in matches]).unique()
locations

array(['000000'], dtype=object)

In [11]:
energies = pd.Series([f[f.find('keV_') - 8 : f.find('keV_')] for f in matches]).unique()
energies

array(['13.45000', '13.45500', '13.46000', '13.46099', '13.46201',
       '13.46301', '13.46401', '13.46501', '13.46550', '13.46599',
       '13.46650', '13.46699', '13.46750', '13.46800', '13.46850',
       '13.46900', '13.46950', '13.47000', '13.47050', '13.47100',
       '13.47150', '13.47200', '13.47251', '13.47300', '13.47350',
       '13.47400', '13.47450', '13.47500', '13.47700', '13.47900',
       '13.48100', '13.48300', '13.48500', '13.49000', '13.49500',
       '13.50000', '13.50500', '13.51000', '13.51500', '13.52000',
       '13.52500', '13.53000', '13.53500', '13.54000', '13.54501',
       '13.55000'], dtype=object)

In [12]:
ids = pd.Series([f[f.find('_id') + 3 : f.find('_SAXS.tif') - 7] for f in matches]).unique()
ids

array(['433726', '433727', '433728', '433729', '433730', '433731',
       '433732', '433733', '433734', '433735', '433736', '433737',
       '433738', '433739', '433740', '433741', '433742', '433743',
       '433744', '433745', '433746', '433747', '433748', '433749',
       '433750', '433751', '433752', '433753', '433754', '433755',
       '433756', '433757', '433758', '433759', '433760', '433761',
       '433762', '433763', '433764', '433765', '433766', '433767',
       '433768', '433769', '433770', '433771', '433772', '433773',
       '433774', '433775', '433776', '433777', '433778', '433779',
       '433780', '433781', '433782', '433783', '433784', '433785',
       '433786', '433787', '433788', '433789', '433790', '433791',
       '433792', '433793', '433794', '433795', '433796', '433797',
       '433798', '433799', '433800', '433801', '433802', '433803',
       '433804', '433805', '433806', '433807', '433808', '433809',
       '433810', '433811', '433812', '433813', '433814', '4338

In [13]:
batch_dct = {}

for sample in samples:
    
    sample_lst = []
    
    # Find data files for each samle and corresponding energies
    to_find = sample # + 'temp'
    matches = files[files.str.contains(f'_{to_find}')]# + r'\d{0,6}.\d{0,2}degC')]
    if matches.empty:
        matches = files[files.str.contains(sample)]## + '16.10keV', regex=False)]
    
    #incident_angles = pd.Series([f[f.find('_ai') + 3 : f.find('_id')] for f in matches]).unique()
    #locations = pd.Series([f[f.find('_loc') + 4 : f.find('_trs')] for f in matches]).unique()
    #temperatures = pd.Series([f[f.find('degC_') - 5 : f.find('degC_')] for f in matches]).unique()
    #temperatures = pd.Series(matches.str.split('_', expand=True).iloc[:, 2].unique())
    #temperatures = temperatures.str.split('degC', expand=True).iloc[:, 0].unique()
    #locations = pd.Series([f[f.find('_SAXS.tif') - 6 : f.find('_SAXS.tif')] for f in matches]).unique()
    #locations = pd.Series([f[f.find('_loc') + 4 : f.find('_loc') + 5] for f in matches]).unique()
    #locations = pd.Series([f[f.find('loc') + 3 : f.find('loc') + 4] for f in matches]).unique()
    #locations = pd.Series([f[f.find('-x') + 0 : f.find('_16.10keV')] for f in matches]).unique()
    #if 'keV' in matches.all():
    #    energies = pd.Series([f[f.find('eV_') - 5 : f.find('keV_')] for f in matches]).unique()
    #else:
    #    energies = pd.Series([f[f.find('eV_') - 7 : f.find('eV_')] for f in matches]).unique()
    #ids = pd.Series([f[f.find('_id') + 3 : f.find('_SAXS.tif') - 7] for f in matches]).unique()
    #sdds = pd.Series([f[f.find('_sdd') + 4 : f.find('m_loc')] for f in matches]).unique()
    #times = pd.Series([f[f.find('_time') + 5 : f.find('_ai')] for f in matches]).unique()
    #runs = pd.Series([f[f.find('_run') + 4 : f.find('keV') - 6] for f in matches]).unique()
    energies = pd.Series([f[f.find('keV_') - 8 : f.find('keV_')] for f in matches]).unique()

    values = ['']
    values2 = energies
    values3 = ['']
    values4 = ['']
    for val in values:
        measurement_lst = []

        scans = matches[matches.str.contains(f'{val}')]
        
        for val2 in values2:
            scans2 = scans[scans.str.contains(f'{val2}')]
            
            for val3 in values3:
                scans3 = scans2[scans2.str.contains(f'{val3}')]
                
                for val4 in values4:
                    scans4 = scans3[scans3.str.contains(f'{val4}')]

                    if not scans4.empty:
                        measurement_lst.append(scans4.values.tolist())
        
        if measurement_lst:
            sample_lst.append(measurement_lst)

            #try:
            #    #position_lst.append(scans.values.tolist())
            #    sample_lst.append(scans2.values.tolist())
            #except:
            #    pass
            #    #sample_lst.append(position_lst)
    batch_dct[sample] = sample_lst

In [14]:
batch_dct

{'IB_kaptom_pos1_': [[['IB_kaptom_pos1_13.45000keV_wa20.0_sdd1.8m_bpm3.415_id433726_000000_SAXS.tif',
    'IB_kaptom_pos1_13.45000keV_wa40.0_sdd1.8m_bpm3.426_id433772_000000_SAXS.tif'],
   ['IB_kaptom_pos1_13.45500keV_wa20.0_sdd1.8m_bpm3.421_id433727_000000_SAXS.tif',
    'IB_kaptom_pos1_13.45500keV_wa40.0_sdd1.8m_bpm03.42_id433773_000000_SAXS.tif'],
   ['IB_kaptom_pos1_13.46000keV_wa20.0_sdd1.8m_bpm3.412_id433728_000000_SAXS.tif',
    'IB_kaptom_pos1_13.46000keV_wa40.0_sdd1.8m_bpm3.411_id433774_000000_SAXS.tif'],
   ['IB_kaptom_pos1_13.46099keV_wa20.0_sdd1.8m_bpm3.404_id433729_000000_SAXS.tif',
    'IB_kaptom_pos1_13.46099keV_wa40.0_sdd1.8m_bpm3.406_id433775_000000_SAXS.tif'],
   ['IB_kaptom_pos1_13.46201keV_wa20.0_sdd1.8m_bpm3.401_id433730_000000_SAXS.tif',
    'IB_kaptom_pos1_13.46201keV_wa40.0_sdd1.8m_bpm3.403_id433776_000000_SAXS.tif'],
   ['IB_kaptom_pos1_13.46301keV_wa20.0_sdd1.8m_bpm3.398_id433731_000000_SAXS.tif',
    'IB_kaptom_pos1_13.46301keV_wa40.0_sdd1.8m_bpm003.4_id43377

In [15]:
i = 0
sample = samples[i]
print('Total samples: {}'.format(len(batch_dct)))
print('Total values scanned for sample {}: {}'.format(sample, len(batch_dct[sample])))
print('Total measurments for sample {}: {}'.format(sample, len(batch_dct[sample][0])))

Total samples: 11
Total values scanned for sample IB_kaptom_pos1_: 1
Total measurments for sample IB_kaptom_pos1_: 46



<h3>Background / hot pixels</h3>

In [16]:
batch_bkg = None

In [17]:
test = sorted([os.path.join(path_data, f) for f in os.listdir(path_data) if ('test_darks' in f) and ('.tif' in f)])
test.sort(key=lambda x: os.path.getmtime(x), reverse=False)
test = pd.Series([os.path.split(path_to_file)[-1] for path_to_file in test])
test

Series([], dtype: object)

In [18]:
if not test.empty:
    fnames = test
else:
    fnames = files.iloc[1:4].tolist()
#fnames

In [19]:
fnames

['IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45101keV_wa20.0_sdd1.8m_bpm003.4_id433606_000000_SAXS.tif',
 'IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45200keV_wa20.0_sdd1.8m_bpm3.399_id433607_000000_SAXS.tif',
 'IB_LiBr-sol-nosolv-a-nexafs_pos1_13.45300keV_wa20.0_sdd1.8m_bpm3.396_id433608_000000_SAXS.tif']

In [20]:

#fnames = files.iloc[-3:-1].tolist()
        
fig, ax = plt.subplots()
#sname = fnames[0][:fnames[0].find('eV') + 2]
#fig.suptitle(sname, y=1.0)

flatfield = fabio.open(os.path.join(path_data, fnames[0])).data
        
for f in fnames[1:30]:
    flatfield += fabio.open(os.path.join(path_data, f)).data
flatfield = np.divide(flatfield, len(fnames), out=np.zeros_like(flatfield),
                      where=flatfield!=0, casting='unsafe')
        
    
im = ax.imshow(flatfield, vmin=0, vmax=np.percentile(flatfield, 99))

ax.set_xlabel('Pixels')
ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes('top', size='3%', pad='2%')
cb = colorbar(cax, im, orientation='horizontal')
cax.set_xlabel('Intensity (arb. u.)', size=8)
cax.xaxis.set_label_position('top')
cax.xaxis.tick_top()
cax.xaxis.set_tick_params(labelsize=8) 

<IPython.core.display.Javascript object>

In [21]:
img = flatfield.copy()
delta = 150
center = [254, 561.0]
range_y = [int(center[1] - 3.7*  delta), int(center[1] + 1.5 * delta)]
range_x = [int(center[0] - 1.5 * delta), int(center[0] + delta)]
limits = np.asarray([0.1, 25])
mean_img = np.mean(np.concatenate((img[:min(range_y), :].flatten(),
                                   img[max(range_y):, :].flatten(),
                                   img[:, :min(range_x)].flatten(),
                                   img[:, max(range_x):].flatten())), axis=0)
limits = limits * mean_img
mask_bkg = np.empty_like(img, dtype=bool)
mask_bkg[:, :] = False
mask_bkg[:min(range_y), :][img[:min(range_y), :] > max(limits)] = True
mask_bkg[max(range_y):, :][img[max(range_y):, :] > max(limits)] = True
mask_bkg[:, :min(range_x)][img[:, :min(range_x)] > max(limits)] = True
mask_bkg[:, max(range_x):][img[:, max(range_x):] > max(limits)] = True
mask_bkg[img < - 0.1] = True

# Coord: Y, X
mask_bkg[520 : 521 + 1, 546] = True
#mask_bkg[520 : 521 + 1, 546] = True
mask_bkg[520 : 521 + 1, 552 : 554 + 1] = True
#mask_bkg[520 : 522 + 1, 614 : 615 + 1] = True
#mask_bkg[962, 329] = True
#mask_bkg[991, 517] = True
#mask_bkg[1032, 510] = True
#mask_bkg[394, 889] = True
#mask_bkg[384, 908] = True
#mask_bkg[396, 908] = True
#mask_bkg[404, 913] = True
#mask_bkg[406, 916] = True
mask_bkg[145, 733] = True
mask_bkg[372, 462] = True
mask_bkg[189, 571] = True
mask_bkg[236, 142] = True

fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(8, 6))
ax[0].imshow(img*np.logical_not(mask_bkg), vmax=np.percentile(img, 99.0))
ax[1].imshow(img, vmax=np.percentile(img, 99.0))
ax[2].imshow(mask_bkg)
ax[2].scatter(np.where(mask_bkg)[1],
              np.where(mask_bkg)[0], s=0.1, color='r')
for axi, title in zip(ax, ['corrected', 'raw', 'removed pixels']):
    axi.set_title(title, size=10)

<IPython.core.display.Javascript object>

<h3>Test geometry</h3>
Check the processing geometry (beam centre) and other parameters on single frame before processing everything.

In [22]:
sample = samples.iloc[-1]
energy_data_set = batch_dct[sample]
fnames = energy_data_set[0][0]
f = fnames[0]
fnames

['IB_DIPA_RbBr_aq_phase-a_pos1_13.45000keV_wa20.0_sdd1.8m_bpm3.412_id435106_000000_SAXS.tif',
 'IB_DIPA_RbBr_aq_phase-a_pos1_13.45000keV_wa40.0_sdd1.8m_bpm3.426_id435152_000000_SAXS.tif']

In [23]:
energy = float(fnames[0][fnames[0].find('keV') - 8: fnames[0].find('keV')])
wav = 1E-10 * 12.398 / energy
print(energy, wav)

13.45 9.217843866171005e-11


In [35]:
def get_sname_md(f):
    """
    Create metadata for sample short name from the image file name
    """
    #temp = f[f.find('degC_') - 5 : f.find('degC')]
    #ai = f[f.find('_ai') + 3 : f.find('_id')]
    #run = f[f.find('_run') + 4 : f.find('keV') - 6]
    #trs = f[f.find('_trs') + 4 : f.find('_id')]
    #loc = f[f.find('AXS.tif') - 7 : f.find('AXS.tif') - 2]
    #loc = f[f.find('_loc') + 4 : f.find('_loc') + 5]
    #loc = f[f.find('loc') + 3 : f.find('loc') + 4]
    #ids = f[f.find('_id') + 3 : f.find('AXS.tiff') - 15]
    #tme = f[f.find('_time') + 5 : f.find('_time') + 5 + 6]
    #temp = f.split('_')[4]
    #loc = f.split(f'{sample}')[1][: f.find('_16.10') - 9]
    ene = f[f.find('keV_') - 8 : f.find('keV_')]
    bpm = f[f.find('_bpm') + 4 : f.find('_id')]

    name = f'{ene}keV_bpm{bpm}'

    return name

In [36]:
def get_sname(sample):
    sname = sample.translate({ord(c): '' for c in "]['"})
    return sname

In [37]:
sname = get_sname(sample) + get_sname_md(f)
sname

'IB_kaptom_pos1_13.45000keV_bpm3.415'

In [38]:
sample = get_sname(sample)
sample

'IB_kaptom_pos1_'

In [39]:
sdd = float(fnames[0][fnames[0].find('_sdd') + 4 : fnames[0].find('_sdd') + 7])
sdd *= 1000
sdd

1800.0

<h4>DataBroker</h4>
This is only available when running on a beamline machine or nsls2 Jupyter hub, it gets beam centre info from scan metadata but could be specified manually

In [40]:
from databroker import Broker
db = Broker.named('smi')

ProfileNotFound: "Profile 'smi' not found. Found profiles [] from directories [PosixPath('/etc/tiled/profiles'), PosixPath('/Library/Application Support/tiled/profiles'), PosixPath('/Users/ianbillinge/micromamba/envs/smi_py311/etc/tiled/profiles'), PosixPath('/Users/ianbillinge/.config/tiled/profiles'), PosixPath('/Users/ianbillinge/Library/Application Support/tiled/profiles')]."

In [41]:
scan_id = int(fnames[0][fnames[0].find('_id') + 3 : fnames[0].find('_0000')])
saxs_setup = db[scan_id].start['SAXS_setup']
print(scan_id)
print(saxs_setup)
beam_centre = saxs_setup['beam_centre']
bs_kind = saxs_setup['bs']

NameError: name 'db' is not defined

In [42]:
bs_kind = 'pd'
beam_centre = [480, 560]

<h3>Masking hot pixels</h3>
Pilatus1m comes with its own mask for detector modules and some known problematic pixels, however additional routine has been implemented below, it rejects pixels outisde of the beam centre (+- delta in pixels) that are brighter than the maximum limit x mean image intensity.

In [43]:
f = fnames[0]
sname = sample + get_sname_md(f)
print(sname)

if 'keV' in fnames[0]:
    energy = float(fnames[0][fnames[0].find('keV') - 8: fnames[0].find('keV')])
else:
    energy = f[f.find('eV_') - 7 : f.find('eV_')]
    energy = 0.001 * float(energy)

wav = 1E-10 * 12.398 / float(energy)

sdd = float(fnames[0][fnames[0].find('_sdd') + 4 : fnames[0].find('_sdd') + 7])
sdd *= 1000

geometry = 'Transmission'
#geometry = 'Reflection'
bs_kind = bs_kind
if geometry == 'Transmission':
    incident_angle = np.deg2rad(0)
else:
    ai = f[f.find('_ai') + 3 : f.find('_id')]
    incident_angle = np.deg2rad(float(ai))

detector = 'Pilatus1m'

center = beam_centre
if bs_kind == 'pd':
    bs_pos = [[int(center[0]) + 6, int(center[1]) - 5]]
else:
    bs_pos = [[int(center[0]) +5 , 65]]


if batch_bkg:
    try:
        fnames_bkg = batch_bkg[np.around(1000 * energy, decimals=2)]
    except(KeyError):
        energies = np.asarray([float(x) for x in batch_bkg.keys()])
        energy_bkg = energies[np.where(energies < 1000 * energy)[0][-1]]
        fnames_bkg = batch_bkg[energy_bkg]
    flat_imgs = [fabio.open(os.path.join(path_bkg, f)).data for f in fnames_bkg]
    flat_img = np.nanmean(flat_imgs, axis=0)


# Processing geometry for SMI
pr = SMI_beamline.SMI_geometry(geometry = geometry,
                               detector = detector,
                               sdd = sdd,
                               wav = wav,
                               alphai = incident_angle,
                               center = center,
                               bs_pos = bs_pos,
                               bs_kind = bs_kind)

# Open and average SAXS data from different shots
pr.open_data(path_data, [fnames[0]])#, optional_mask='tender')
for f in fnames[1:]:
    pr.imgs[0] += fabio.open(os.path.join(path_data, f)).data
pr.imgs[0] = pr.imgs[0] / len(fnames)


plt.figure()
for i, (img, mask) in enumerate(zip(pr.imgs, pr.masks)):
    
    if batch_bkg:
        pr.imgs[0] = np.divide(pr.imgs[0], flat_img,
                               out=np.zeros_like(pr.imgs[0]),
                               where=(flat_img > 0),
                               casting='unsafe')
        
        #pr.imgs[0] = pr.imgs[0] - flat_img
        
    if bs_kind != 'rod':
        mask[bs_pos[0][1]:, bs_pos[0][0] - 15 : bs_pos[0][0] + 6] = True
        xx, yy = np.mgrid[:1043, :981]
        dia = 23
        circle = (xx - bs_pos[0][1]) ** 2 + (yy - bs_pos[0][0]) ** 2
        mask_circ = (circle < (dia ** 2))
        mask[mask_circ] = True
        
    mask[img < 0] = True
    mask[mask_bkg] = True
    

    pr.masks[i] = mask
    vmax = np.percentile(img*np.logical_not(mask), 99.9)
    plt.imshow(img*np.logical_not(mask), vmin = 0, vmax=vmax)
plt.show()

fig, ax = plt.subplots()
ax.set_title(f'{sample}: masked hot pixels')
ax.imshow(img, vmax=vmax)
#ax.scatter(np.where(img[:min(range_y), :] > max(limits))[1],
#           np.where(img[:min(range_y), :] > max(limits))[0])
#ax.scatter(np.where(img[max(range_y):, :] > max(limits))[1],
#           np.where(img[max(range_y):, :] > max(limits))[0] + max(range_y))
#ax.scatter(np.where(img[:, :min(range_x)] > max(limits))[1],
#           np.where(img[:, :min(range_x)] > max(limits))[0])
#ax.scatter(np.where(img[:, max(range_x):] > max(limits))[1] + max(range_x),
#           np.where(img[:, max(range_x):] > max(limits))[0])
ax.scatter(np.where(mask_bkg)[1],
           np.where(mask_bkg)[0])
#-15_loc2_trs0.92019 -> what's this??

pr.stitching_data(interp_factor=1)

plt.figure()
plt.imshow(pr.img_st,
           extent=[pr.qp[0], pr.qp[-1], pr.qz[0], pr.qz[-1]], 
           vmin=0,
           vmax=np.percentile(pr.img_st, 99.9))    
plt.title('saxs_%s'%sample)
plt.xlabel('$q_{par}(A^{-1})$')
plt.ylabel('$q_{ver}(A^{-1})$')
plt.colorbar()
plt.show()

pr.caking()
plt.figure()
plt.imshow(pr.cake,
           extent=[pr.q_cake[0], pr.q_cake[-1], pr.chi_cake[0], pr.chi_cake[-1]], 
           vmin=0,
           vmax=np.percentile(pr.cake, 95),
           aspect='auto')    
plt.title('saxs_%s'%sample)
plt.xlabel('$q_{par}(A^{-1})$')
plt.ylabel('$q_{ver}(A^{-1})$')
plt.colorbar()
plt.show()


pr.radial_averaging(radial_range = [0.001, pr.q_cake[-1]], azimuth_range=[-90, 0], npt = 500)
q0, I0 = pr.q_rad, pr.I_rad

pr.radial_averaging(radial_range = [0.001, pr.q_cake[-1]], azimuth_range=[-179, 179], npt = 500)
q1, I1 = pr.q_rad, pr.I_rad

pr.radial_averaging(radial_range = [0.001, pr.q_cake[-1]], azimuth_range=[0, 90], npt=500)
q2, I2 = pr.q_rad, pr.I_rad
pr.radial_averaging(radial_range = [0.001, pr.q_cake[-1]], azimuth_range=[90, 179], npt=500)
q3, I3 = pr.q_rad, pr.I_rad
pr.radial_averaging(radial_range = [0.001, pr.q_cake[-1]], azimuth_range=[-90, -179], npt=500)
q4, I4 = pr.q_rad, pr.I_rad

plt.figure()
plt.plot(q0, I0, linewidth = 3, label = 'top right')
plt.plot(q1, I1, linewidth = 3, label='total')
plt.plot(q2, I2, linewidth = 3, label='bottom right')
plt.plot(q3, I3, linewidth = 3, label='bottom left')
plt.plot(q4, I4, linewidth = 3, label='top left')

plt.xscale('log')
plt.yscale('log')
plt.show()
plt.legend()

if pr.geometry == 'Reflection':
    #Figure 4 Plotting - Radial Averaging
    pr.horizontal_integration(q_per_range=[0, pr.q_cake[-1]])
    q0, I0 = pr.q_hor, pr.I_hor

    plt.figure()
    plt.plot(q0, I0, linewidth = 3, color = 'k')

    #plt.yscale('log')
    plt.show()

print('Done')

IB_kaptom_pos1_13.45000keV_bpm3.415


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done


In [44]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True)
img = pr.imgs[0]
mask = pr.masks[0]
# mask[1032, 510] = True
# mask[394, 889] = True
# mask[384, 908] = True
# mask[396, 908] = True
# mask[404, 913] = True
# mask[406, 916] = True

ax[0].imshow(img*np.logical_not(mask), vmax=np.percentile(img, 99.0))
ax[1].imshow(img, vmax=np.percentile(img, 99.0))

<IPython.core.display.Javascript object>

In [45]:
fig, ax = plt.subplots()
ax.set_title(f'{sample}: masked hot pixels')
ax.imshow(img, vmax=vmax)# * 0.01)
#ax.scatter(np.where(img[:min(range_y), :] > max(limits))[1],
#           np.where(img[:min(range_y), :] > max(limits))[0])
#ax.scatter(np.where(img[max(range_y):, :] > max(limits))[1],
#           np.where(img[max(range_y):, :] > max(limits))[0] + max(range_y))
#ax.scatter(np.where(img[:, :min(range_x)] > max(limits))[1],
#           np.where(img[:, :min(range_x)] > max(limits))[0])
#ax.scatter(np.where(img[:, max(range_x):] > max(limits))[1] + max(range_x),
#           np.where(img[:, max(range_x):] > max(limits))[0])
ax.scatter(np.where(mask)[1],
           np.where(mask)[0], s=0.1)

<IPython.core.display.Javascript object>

<h3>Saving mask for IgorPro</h3>

<h2>Batch processing</h2>
Once the geometry has been refined (finding the centre of the beam position on the detector and perhaps refining the beam stop, all the data can be processed.

In [46]:
batch_bkg_var = False

In [ ]:
flag_save = True       # save data / False for test visualisation
cake_save = False       # save tiff with 2D cake data
cake_save_png = False  # save png with 2D cake data
twodim_save = False     # save tiff with 2D q map 
twodim_save_png = True # save png with 2D q map 
hor_ver_save = False    # save horizontal and vertical integration profiles
onedim_save_png = False # save 1d profile as png figure
horref_save_png = True # save 1d horizontal cut in reflection geometry profile as png figure
start = time.time()

# Do not showi figures while processing
plt.ioff() if flag_save else plt.ion()
idx = None if flag_save else 1

for batch_bkg_var in [None]:#, batch_bkg]:
    # Do not showi figures while processing
    plt.ioff() if flag_save else plt.ion()

    # For masking hot pixels got to line 52
    
    for sample in samples[:idx]:


        sample_data = batch_dct[sample]
        print(sample)
        if flag_save:
            dir_name = get_sname(sample)
            save_dir = os.path.join(path_save, dir_name)
            os.makedirs(save_dir, exist_ok=True)

        for data_set in sample_data[:idx]:

            for i, fnames in enumerate(data_set[:idx]):
                fnames = fnames

                # Check if processed data exists and if not, run processing
                f = fnames[0]
                #sname = sample + get_sname_md(f)
                sname = get_sname(sample) + get_sname_md(f)

                if flag_save:
                    save_path = os.path.join(save_dir, f'{sname}_radialint_tot.txt')
                    tail = os.path.split(save_path)[1]
                    condition = tail not in os.listdir(save_dir)
                else:
                    condition = True

                #if True: #process all
                if condition:

                    print(sname)

                    if 'keV' in f:
                            energy = float(f[f.find('keV') - 8: f.find('keV')])
                    else:
                        energy = f[f.find('eV_') - 7 : f.find('eV_')]
                        energy = 0.001 * float(energy)
                    wav = 1E-10 * 12.398 / float(energy)

                    sdd = float(f[f.find('_sdd') + 4 : f.find('_sdd') + 7])
                    sdd *= 1000
                    if geometry == 'Reflection':
                        ai = f[f.find('_ai') + 3 : f.find('_id')]
                        incident_angle = np.deg2rad(float(ai))
                        #sname = sname + f'_ai{ai}'
                    else:
                        incident_angle = np.deg2rad(0)
                        
                    if flag_save == False:
                        print(f'Energy {energy} keV\nsdd {sdd} mm')


                    # Load files for background correction
                    if batch_bkg_var:
                        try:
                            fnames_bkg = batch_bkg[np.around(1000 * energy, decimals=2)]
                        except(KeyError):
                            energies = np.asarray([float(x) for x in batch_bkg.keys()])
                            energy_bkg = energies[np.where(energies < 1000 * energy)[0][-1]]
                            fnames_bkg = batch_bkg[energy_bkg]
                        flat_imgs = [fabio.open(os.path.join(path_bkg, f)).data for f in fnames_bkg]
                        flat_img = np.nanmean(flat_imgs, axis=0)

                    # Processing geometry for SMI
                    pr = SMI_beamline.SMI_geometry(geometry = geometry,
                                                   detector = detector,
                                                   sdd = sdd,
                                                   wav = wav,
                                                   alphai = incident_angle,
                                                   center = center,
                                                   bs_pos = bs_pos,
                                                   bs_kind = bs_kind)

                    # Open and average SAXS data from different waxs angle shots
                    pr.open_data(path_data, [fnames[0]])#, optional_mask='tender')
                    for f in fnames[1:]:
                        pr.imgs[0] += fabio.open(os.path.join(path_data, f)).data
                    pr.imgs[0] = pr.imgs[0] / len(fnames)

                    for i, (img, mask) in enumerate(zip(pr.imgs, pr.masks)):

                        if batch_bkg_var:
                            pr.imgs[0] = np.divide(pr.imgs[0], flat_img,
                                                   out=np.zeros_like(pr.imgs[0]),
                                                   where=(flat_img > 0),
                                                   casting='unsafe')
                            
                        if bs_kind != 'rod':
                            mask[bs_pos[0][1]:, bs_pos[0][0] - 15 : bs_pos[0][0] + 6] = True
                            xx, yy = np.mgrid[:1043, :981]
                            dia = 23
                            circle = (xx - bs_pos[0][1]) ** 2 + (yy - bs_pos[0][0]) ** 2
                            mask_circ = (circle < (dia ** 2))
                            mask[mask_circ] = True
                        mask[img < -1] = True
                        mask[mask_bkg] = True  
                        pr.masks[i] = mask

                    # Stitch data
                    pr.stitching_data(interp_factor=1)

                    # Plot and save stitching data
                    fig, ax = plt.subplots()
                    #im = ax.imshow(pr.img_st,
                    #               extent=[pr.qp[0], pr.qp[-1], pr.qz[0], pr.qz[-1]], 
                    #               vmin=0, vmax=np.percentile(pr.img_st, 99),
                    #               #norm = colors.LogNorm(vmin=50,
                    #               #                      #vmax=np.percentile(pr.img_st, 99.999),
                    #               #                      vmax = np.nanmax(pr.img_st),
                    #               #                      clip=True),
                    #               cmap=cmap,
                    #              )
                    vmin, vmax = np.nanpercentile(pr.img_st, (50, 99.90))
                    if (vmin < 0.01) or (np.isnan(vmin)) or (np.isinf(vmin)):
                        vmin = 10
                    if (vmax < 0.01) or (np.isnan(vmax)) or (np.isinf(vmax)):
                        vmax = 100
                    norm = colors.LogNorm(vmin=vmin, vmax=vmax, clip=True)
                    im = ax.imshow(pr.img_st,
                                   extent=[pr.qp[0], pr.qp[-1], pr.qz[0], pr.qz[-1]], 
                                   #vmin=0, vmax=np.percentile(pr.img_st, 99),
                                   norm=norm, cmap=cmap)
                    ax_divider = make_axes_locatable(ax)
                    cax = ax_divider.append_axes('right', size='3%', pad='2%')
                    cb = colorbar(cax, im)
                    cax.set_ylabel('Intensity (arb. u.)', labelpad=8)
                    ax.xaxis.set_minor_locator(MultipleLocator(0.1))

                    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
                    if pr.geometry == 'Transmission':
                        ax.set_xlabel('$q_{\mathrm{hor}}$ $({\mathrm{\AA}}^{-1})$')
                        ax.set_ylabel('$q_{\mathrm{ver}}$ $({\mathrm{\AA}}^{-1})$')
                    else:
                        ax.set_xlabel('$q_{\mathrm{xy}}$ $({\mathrm{\AA}}^{-1})$')
                        ax.set_ylabel('$q_{\mathrm{z}}$ $({\mathrm{\AA}}^{-1})$')
                    ax.set_title('{}'.format(sname))

                    if flag_save:
                        
                        if twodim_save_png:
                            fname = '{}_qmap'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.png')
                            fig.savefig(save_path, bbox_inches='tight', dpi=300)

                        if twodim_save:
                            fname = '{}_qmap_Intensity'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.tif')
                            fabio.tifimage.TifImage(np.nan_to_num(pr.img_st,).astype('int')).write(save_path)

                            fname = '{}_qmap_qhor'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.txt')
                            qp = np.linspace(pr.qp[0], pr.qp[1], pr.img_st.shape[1])
                            np.savetxt(save_path, qp, fmt='%1.6f', header='q_hor (A-1)')

                            fname = '{}_qmap_qver'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.txt')
                            qp = np.linspace(pr.qz[0], pr.qz[1], pr.img_st.shape[1])
                            np.savetxt(save_path, qp, fmt='%1.6f', header='q_ver (A-1)')

                    # Cake data
                    pr.caking(radial_range = (0.001, np.sqrt(pr.qp[1] ** 2 + pr.qz[1] ** 2)))
                    
                    fig, ax = plt.subplots()
                    vmin, vmax = np.nanpercentile(pr.cake, (50, 99.99))
                    if np.isnan(vmin) or (vmin < 1):
                        vmin = 0.1
                    norm = colors.LogNorm(vmin=vmin, vmax=vmax, clip=True)    

                    if np.isnan(vmax) or (vmax < 1):
                        norm = None

                    im = ax.imshow(pr.cake,
                                   extent=[pr.q_cake[0], pr.q_cake[-1],
                                           pr.chi_cake[-1], pr.chi_cake[0]], 
                                   #vmin=0, vmax=vmax,
                                   norm=norm,
                                   aspect='auto', origin='lower', cmap=cmap)

                    sci_formatter = ScalarFormatter(useMathText=True)
                    sci_formatter.set_scientific(True)
                    sci_formatter.set_powerlimits((-1, 1))
                    ax_divider = make_axes_locatable(ax)
                    cax = ax_divider.append_axes('right', size='3%', pad='2%')

                    if norm == None:
                        mappable = im
                    else:
                        mappable = cm.ScalarMappable(
                            norm=colors.LogNorm(vmin=vmin, vmax=vmax, clip=True),
                            cmap=cmap)

                    cb = colorbar(cax, mappable)
                    #cb = colorbar(cax, im)#, format=sci_formatter)
                    cax.set_ylabel('Intensity (arb. u.)', labelpad=8)
                    cax.yaxis.set_offset_position('left')
                    #ax.xaxis.set_minor_locator(MultipleLocator(0.1))
                    ax.yaxis.set_ticks(np.arange(-180, 180, 45))
                    #ax.yaxis.set_minor_locator(MultipleLocator(30))
                    ax.set_xlabel('$q$ $({\mathrm{\AA}}^{-1})$')
                    ax.set_ylabel('$\chi$ (deg.)')
                    ax.set_title('{}'.format(sname))
                    #ax.set_xscale('log')

                    if flag_save:
                        
                        if cake_save_png:
                            fname = '{}_cake'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.png')
                            fig.savefig(save_path, bbox_inches='tight', dpi=300)
                        
                        if cake_save:

                            fname = '{}_cake_Intensity'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.tif')
                            fabio.tifimage.TifImage(np.nan_to_num(pr.cake).astype('int')).write(save_path)

                            if pr.geometry == 'Transmission':
                                fname = '{}_cake_chi_q'.format(sname)
                                save_path = os.path.join(save_dir, fname + '.txt')  
                                np.savetxt(save_path, np.vstack((pr.chi_cake, pr.q_cake)).T,
                                           fmt='%1.2f\t%1.6f', header='chi (deg)\tq (A-1)')
                            else:
                                fname = '{}_cake_chi'.format(sname)
                                save_path = os.path.join(save_dir, fname + '.txt')  
                                np.savetxt(save_path, pr.chi_cake.T, fmt='%1.2f', header='chi (deg)')

                                fname = '{}_cake_q'.format(sname)
                                save_path = os.path.join(save_dir, fname + '.txt')  
                                np.savetxt(save_path, pr.q_cake.T, fmt='%1.6f', header='q (A-1)')

                    # Radial average

                    param_rad = dict(radial_range=[0.0005, pr.q_cake[-5]], npt=2000)
                    if pr.geometry == 'Transmission':
                        azimuth_ranges = dict(tot=[-179.9, 179.9],
                                              hor=[-179.9, -170],
                                              ver=[-95, -85])
                    else:
                        azimuth_ranges = dict(tot=[-90, 90],
                                              hor=[45, 135],
                                              ver=[-45, 45])
                    fig, ax = plt.subplots()
                    params = dict(ls='-', lw=1, ms=5, mfc='w', marker='o', mew=1.1, alpha=1)
                    
                    pr.radial_averaging(azimuth_range=azimuth_ranges['tot'], **param_rad)
                    q_tot, I_tot = pr.q_rad, pr.I_rad
                    ax.plot(q_tot, I_tot,
                            label='$\chi$: ' + '{}, tot'.format(azimuth_ranges['tot']), **params)
                    
                    if hor_ver_save:
                        pr.radial_averaging(azimuth_range=azimuth_ranges['hor'], **param_rad)
                        q_hor, I_hor = pr.q_rad, pr.I_rad
                        pr.radial_averaging(azimuth_range=azimuth_ranges['ver'], **param_rad)
                        q_ver, I_ver = pr.q_rad, pr.I_rad

                        ax.plot(q_hor, I_hor,
                                label='$\chi$: ' + '{}, hor'.format(azimuth_ranges['hor']), **params)
                        ax.plot(q_ver, I_ver,
                                label='$\chi$: ' + '{}, ver'.format(azimuth_ranges['ver']), **params)
                    ax.set_xlabel('$q$ $({\mathrm{\AA}}^{-1})$')
                    ax.set_ylabel('Intensity (arb. u.)')
                    ax.set_yscale('log')
                    ax.legend()
                    ax.grid(which='major', linestyle='--', lw=0.5)
                    ax.grid(which='minor', linestyle='--', lw=0.2)
                    #ax.xaxis.set_minor_locator(MultipleLocator(0.01))
                    #ax.xaxis.set_major_locator(MultipleLocator(0.01))
                    ax.set_title('{}'.format(sname))
                    ax.set_xscale('log')

                    if flag_save:
                         
                        if onedim_save_png:
                            fname = '{}_radialint'.format(sname)
                            save_path = os.path.join(save_dir, fname + '.png')
                            fig.savefig(save_path, bbox_inches='tight', dpi=300)

                        fname = '{}_radialint_'.format(sname)
                        save_path = os.path.join(save_dir, fname + 'tot.txt')  
                        np.savetxt(save_path, np.vstack((q_tot, I_tot)).T,
                                   fmt='%1.6f\t%1.1f', header='q (A-1)\tI (arb. u.)')
                        if hor_ver_save:
                            save_path = os.path.join(save_dir, fname + 'hor.txt')  
                            np.savetxt(save_path, np.vstack((q_hor, I_hor)).T,
                                       fmt='%1.6f\t%1.1f', header='q (A-1)\tI (arb. u.)')
                            save_path = os.path.join(save_dir, fname + 'ver.txt')  
                            np.savetxt(save_path, np.vstack((q_ver, I_ver)).T,
                                       fmt='%1.6f\t%1.1f', header='q (A-1)\tI (arb. u.)')
                        plt.close(fig='all')

                    # Horizontal integration
                    if pr.geometry == 'Reflection':

                        pr.horizontal_integration(q_per_range=[0.05, 0.06])

                        fig, ax = plt.subplots()
                        params = dict(ls='None', lw=0.5, ms=5, mfc='w', marker='o', mew=1.1, alpha=1, c='k')
                        #ax.plot(pr.q_hor, pr.I_hor, lw=2, c='k', label='Horizontal profile')
                        ax.plot(pr.q_hor, pr.I_hor, label='Horizontal profile', **params)
                        ax.set_xlabel('$q$ $({\mathrm{\AA}}^{-1})$')
                        ax.set_ylabel('Intensity (arb. u.)')
                        ax.set_yscale('log')
                        ax.legend()
                        ax.grid(which='major', linestyle='--', lw=0.5)
                        #ax.xaxis.set_minor_locator(MultipleLocator(0.001))
                        #ax.xaxis.set_major_locator(MultipleLocator(0.01))
                        ax.set_title('{}'.format(sname))

                        if flag_save:
                            fname = '{}_horizontal_profile'.format(sname)
                            
                            if horref_save_png:
                                save_path = os.path.join(save_dir, fname + '.png')
                                fig.savefig(save_path, bbox_inches='tight', dpi=300)

                            save_path = os.path.join(save_dir, fname + '.txt')  
                            np.savetxt(save_path, np.vstack((pr.q_hor, pr.I_hor)).T,
                                       fmt='%1.6f\t%1.1f', header='q (A-1)\tI (arb. u.)')
                            plt.close(fig='all')
stop = time.time()
print('Processing time: {:.1f} minutes'.format((stop - start) / 60))
print('Processing time: {:.1f} hours'.format((stop - start) / 60 / 60))

IB_kaptom_pos1_
IB_kaptom_pos1_13.45000keV_bpm3.415


IB_kaptom_pos1_13.45500keV_bpm3.421


IB_kaptom_pos1_13.46000keV_bpm3.412


IB_kaptom_pos1_13.46099keV_bpm3.404


IB_kaptom_pos1_13.46201keV_bpm3.401


...

IB_DIPA_RbBr_aq_phase-a_pos1_13.47300keV_bpm3.365


IB_DIPA_RbBr_aq_phase-a_pos1_13.47350keV_bpm3.356


IB_DIPA_RbBr_aq_phase-a_pos1_13.47400keV_bpm3.355


IB_DIPA_RbBr_aq_phase-a_pos1_13.47450keV_bpm3.359


IB_DIPA_RbBr_aq_phase-a_pos1_13.47500keV_bpm03.35


<h3>Check file loading</h3>

In [ ]:
plt.ion()
print(save_dir)
print(sname)

In [ ]:
fname = '{}_qmap_Intensity'.format(sname)
save_path = os.path.join(save_dir, fname + '.tif')
img = fabio.open(save_path).data

fname = '{}_qmap_qhor'.format(sname)
save_path = os.path.join(save_dir, fname + '.txt')
q_hor = np.loadtxt(save_path, comments='#', delimiter=None, skiprows=1)

fname = '{}_qmap_qver'.format(sname)
save_path = os.path.join(save_dir, fname + '.txt')
q_ver = np.loadtxt(save_path, comments='#', delimiter=None, skiprows=1)

vmax = np.nanpercentile(img, 97)
fig, ax = plt.subplots()
im = ax.imshow(np.log(img), extent=[q_hor[0], q_hor[-1], q_ver[0], q_ver[-1]])
plt.colorbar(im, ax=ax)